# E6 인공지능 작사가 만들기

## ?? 프로그램 목적 : 인공지능으로 멋진 작사 해 보기
            
### (정규표현식 - 데이터 다듬기, Embadding, LSTM으로 학습 시키기, 잘 만들어 졌는지 평가하기)

### Step 1,2. 데이터 준비하기

In [1]:
import glob
import os
import re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        print(txt_file)
        print("데이터 크기:", len(raw))
        print("Examples:\n", raw[:5])
        print("\n")
        raw_corpus.extend(raw)
        

AttributeError: module 'tensorflow.python.keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

## !! 폴더에 들어 있는 각 파일의 형태를 보고 싶어서 5개씩 출력 해 보니 제거해야할 부분이 보인다

## Step 3. 데이터 정제
### Key point : preprocess_sentence() 함수 사용, 지나치게 긴 문장은  제거,  토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기

In [19]:
# 문장 다듬는 preprocess_sentence 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    return sentence

# 문장길이가 100이상으로 너무 길면 제외
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) > 100: continue   # 길이가 100인 문장은 건너뜁니다.
        
    corpus.append(preprocess_sentence(sentence))

# 10개 정도 출력 해보자
print("제거전 문장수 : ", len(raw_corpus))
print("제거후 문장수 : ", len(corpus))

corpus[:10]


제거전 문장수 :  187088
제거후 문장수 :  174176


['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

### 자 이제 토큰화, 토큰 갯수가 15개 넘어가면 제외를 시켜야 하는데...

In [26]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  900 ...    0    0    0]
 [   2  120 2638 ...    0    0    0]
 [   2    6  300 ...    0    0    0]
 ...
 [   2  135    4 ...    0    0    0]
 [   2  122  169 ...    0    0    0]
 [   2  135    4 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f829c2bb950>


In [22]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

BUFFER_SIZE = len(src_input)
print(BUFFER_SIZE)

BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

174176


<BatchDataset shapes: ((256, 34), (256, 34)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

In [30]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [31]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 34, 7001), dtype=float32, numpy=
array([[[ 2.2714717e-05,  9.3209092e-06, -3.8850329e-05, ...,
         -1.4518519e-05, -2.0055540e-04, -1.6173880e-04],
        [-6.8089998e-05,  3.5536863e-04, -2.0061476e-04, ...,
         -1.8627479e-04, -1.4829457e-04, -3.4798915e-04],
        [-3.9664880e-04,  6.4203935e-04, -3.4751196e-05, ...,
         -6.1696017e-04, -4.7002523e-04, -5.6292349e-04],
        ...,
        [ 1.2543742e-04, -2.5719584e-03, -1.1802578e-03, ...,
          1.6681366e-03,  7.7560153e-03, -3.4933144e-03],
        [ 1.4082209e-04, -2.5796329e-03, -1.1783527e-03, ...,
          1.6791236e-03,  7.7779354e-03, -3.5217134e-03],
        [ 1.5351159e-04, -2.5842031e-03, -1.1762910e-03, ...,
          1.6873002e-03,  7.7934917e-03, -3.5441434e-03]],

       [[ 2.2714717e-05,  9.3209092e-06, -3.8850329e-05, ...,
         -1.4518519e-05, -2.0055540e-04, -1.6173880e-04],
        [-1.7308535e-04, -1.5968810e-04,  1.1668187e-04, ...,
         -2.3020169e-05, -

In [32]:
# 여기부터 학습인데 많이 걸린다....

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
 11/680 [..............................] - ETA: 38:30 - loss: 6.0814

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)